In [32]:
from math import sqrt


# Funções básicas

## Calculo do erro

In [33]:
def erro_absoluto(referencia, estimativa):
    return abs(referencia - estimativa)


def erro_relativo(referencia, estimativa):
    return abs((referencia, estimativa) / estimativa)


# Derivação numerica

In [34]:
DELTA_DERIVACAO_PADRAO = 10**-4


def derivada_direita(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x)) / delta


def derivada_esquerda(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x - delta) - f(x)) / delta


def derivada_central(f, delta=DELTA_DERIVACAO_PADRAO):
    return lambda x: (f(x + delta) - f(x - delta)) / (2 * delta)


# Métodos iterativos

## Condições de parada

In [35]:
def limite_iteracao(i):
    return lambda est: est.i >= i


def criterio_erro_absoluto(referencia, e):
    return lambda est: erro_absoluto(referencia, est.x) < e


def criterio_ab(e):
    return lambda est: est.b - est.a < 2 * e


def criterio_x(e):
    return lambda est: est.f(est.x - e) * est.f(est.x + e) < 0


def criterio_delta_x(delta):
    return lambda est: est.prior_x is not None and abs(est.x - est.prior_x) < delta


def criterio_y(e):
    return lambda est: abs(est.y) < e


## Método da bisseçao

In [36]:
class EstatisticaAB:
    def __init__(self, f, a, b, x, y):
        self.f = f
        self.i = 1
        self.a = a
        self.b = b
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, a, b, x, y):
        self.i += 1
        self.a = a
        self.b = b
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, a: {self.a}, b: {self.b}, x: {self.x}, y: {self.y}"


In [37]:
def bissecao(f, a, b, criterio, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a + b) / 2
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a + b) / 2
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método da falsa posição

In [38]:
def falsa_posicao(f, a, b, criterio, observador=lambda _: None):
    if a > b:
        a, b = b, a
    i = 1
    x = (a * f(b) - b * f(a)) / (f(b) - f(a))
    y = f(x)
    est = EstatisticaAB(f, a, b, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        if f(a) * y < 0:
            b = x
        else:
            a = x
        x = (a * f(b) - b * f(a)) / (f(b) - f(a))
        y = f(x)
        est_erro = (b - a) / 2
        est.atualizar(a, b, x, y)
        observador(est)
    return est


## Método do ponto fixo

In [39]:
class EstatisticaX:
    def __init__(self, f, fi, x, y) -> None:
        self.i = 1
        self.f = f
        self.fi = fi
        self.prior_x = None
        self.x = x
        self.y = y

    def atualizar(self, x, y):
        self.i += 1
        self.prior_x = x
        self.x = x
        self.y = y

    def __str__(self) -> str:
        return f"iteracao: {self.i}, x: {self.x}, y:{self.y}"


In [40]:
def ponto_fixo(f, fi, x, criterio, observador=lambda _: None):
    i = 1
    y = f(x)
    est = EstatisticaX(f, fi, x, y)
    observador(est)
    while not criterio(est):
        i += 1
        x = fi(x)
        y = f(x)
        est.atualizar(x, y)
        observador(est)
    return est


## Método de Newtown-Raphson

In [41]:
def newton_raphson(f, x, criterio, observador=lambda x: None):
    d = derivada_central(f)
    fi = lambda x: x - f(x) / d(x)
    return ponto_fixo(f, fi, x, criterio, observador)


# Amostras

In [42]:
def amostra_linear(x):
    return 5 * x + 2


def amostra_poli(x):
    return x**2 + x - 6


def amostra_poli_iter(x):
    return sqrt(6 - x)


# Playground 

In [43]:
from math import cos, log, exp


In [44]:
def f_q3(x):
    return 1 / x - exp(-3 * x) - 0.13

def criterio_q3(ex, ey):
    cy = criterio_y(ey)
    cx = criterio_x(ex)
    return lambda est: cx(est) and cy(est)

def f_q4(x):
    return log(x**2 + 1) - cos(x)


def f_q5(x):
    return x**3 - 3 * x**2 - 3 * x * log(x)


def d_q5(x):
    return 3 * x**2 - 6 * x - 3 * log(x) - 3


def fi_q5(x):
    return x - f_q5(x) / d_q5(x)


In [45]:
# q3b
resultado = bissecao(f_q3, 7.2, 7.8, criterio_q3(10 ** -2, 10 ** -4), print)

iteracao: 1, a: 7.2, b: 7.8, x: 7.5, y: 0.003333333164143526
iteracao: 2, a: 7.5, b: 7.8, x: 7.65, y: 0.0007189541404858391
iteracao: 3, a: 7.65, b: 7.8, x: 7.725, y: -0.0005501618984417866
iteracao: 4, a: 7.65, b: 7.725, x: 7.6875, y: 8.130071660669702e-05


In [46]:
# q4
resultado = falsa_posicao(f_q4, 0.0, 2.0, criterio_x(2 * 10**-4), print)


iteracao: 1, a: 0.0, b: 2.0, x: 0.6610292442389619, y: -0.4268312306443269
iteracao: 2, a: 0.6610292442389619, b: 2.0, x: 0.894070677758534, y: -0.038811244777121945
iteracao: 3, a: 0.894070677758534, b: 2.0, x: 0.9148624704966931, y: -0.001780277110784767
iteracao: 4, a: 0.9148624704966931, b: 2.0, x: 0.9158153553896283, y: -7.569069194346412e-05


In [47]:
# q5 - raiz proxima de 1
resultado = ponto_fixo(f_q5, fi_q5, 1, criterio_x(10 ** - 7), print)


iteracao: 1, x: 1, y:-2.0
iteracao: 2, x: 0.6666666666666667, y:-0.22610682082070865
iteracao: 3, x: 0.615859243157585, y:-0.008674126976297214
iteracao: 4, x: 0.61374519936385, y:-1.605754214561994e-05
iteracao: 5, x: 0.6137412712931709, y:-5.559019911061114e-11


In [48]:
# q5 - raiz proxima de 5
resultado = ponto_fixo(f_q5, fi_q5, 5, criterio_x(10 ** -7), print)

iteracao: 1, x: 5, y:25.858431313488495
iteracao: 2, x: 4.304351405240451, y:5.318058043063292
iteracao: 3, x: 4.066697636897585, y:0.5264052300437818
iteracao: 4, x: 4.037461687870278, y:0.007522675794341183
iteracao: 5, x: 4.037031616014951, y:1.6166440381937264e-06
